# Support Vector Machines

**SVM** (Support Vector Machine) operates in a versatile way, being able to perform linear or non-linear classification, regression and novelty detection. It does not scale well with very large datasets.

# Linear SVM Classification

## Soft Margin Classification

# Nonlinear SVM Classification

## Polynomial Kernel

## Similarity Features

## Gaussian RBF Kernel

## SVM Classes and Computational Complexity

# SVM Regression

# Under the Hood of Linear SVM Classifiers

# The Dual Problem 

## Kernelized SVMs 